## Data cleaning

In [4]:
import pandas as pd
import numpy as np
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
import missingno as msno
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [5]:
# Load the csv file
csv = pd.read_csv("bicycle_sharing.csv", low_memory=False)
csv.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,Year
0,A847FADBBC638E45,docked_bike,2020-04-26 17:45:14,2020-04-26 18:12:03,Eckhart Park,86,Lincoln Ave & Diversey Pkwy,152.0,41.90,-87.66,41.93,-87.66,member,2020
1,5405B80E996FF60D,docked_bike,2020-04-17 17:08:54,2020-04-17 17:17:03,Drake Ave & Fullerton Ave,503,Kosciuszko Park,499.0,41.92,-87.72,41.93,-87.72,member,2020
2,5DD24A79A4E006F4,docked_bike,2020-04-01 17:54:13,2020-04-01 18:08:36,McClurg Ct & Erie St,142,Indiana Ave & Roosevelt Rd,255.0,41.89,-87.62,41.87,-87.62,member,2020
3,2A59BBDF5CDBA725,docked_bike,2020-04-07 12:50:19,2020-04-07 13:02:31,California Ave & Division St,216,Wood St & Augusta Blvd,657.0,41.90,-87.70,41.90,-87.67,member,2020
4,27AD306C119C6158,docked_bike,2020-04-18 10:22:59,2020-04-18 11:15:54,Rush St & Hubbard St,125,Sheridan Rd & Lawrence Ave,323.0,41.89,-87.63,41.97,-87.65,casual,2020


In [6]:
csv.info(memory_usage="deep")
### Memory usage is very high

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14758322 entries, 0 to 14758321
Data columns (total 14 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
 13  Year                int64  
dtypes: float64(4), int64(1), object(9)
memory usage: 9.2 GB


In [8]:
# Percent of missing data, sorted descending order
(csv.apply(pd.isnull).sum()/csv.shape[0] * 100).sort_values(ascending=False)

end_station_id       12.18
end_station_name     12.18
start_station_id     11.33
start_station_name   11.32
end_lat               0.10
end_lng               0.10
ride_id               0.00
rideable_type         0.00
started_at            0.00
ended_at              0.00
start_lat             0.00
start_lng             0.00
member_casual         0.00
Year                  0.00
dtype: float64